In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import project_path

In [3]:
import glob
import os

from pathlib import Path
from itertools import product

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from src.plotting import make_result_heatmap, add_lines

In [4]:
pd.read_csv('../experiments.csv')

,id,Type of experiment
0,0,Local experiments
1,1590579968,SBM
2,1590672352,SBM test nb cuts
3,1590679038,SBM 20 cuts keep 80%
4,1590585263,Mindsets (old)
5,1590663805,Mindsets
6,1590669671,Mindsets better resolution
7,1590676940,Mindsets better resolution 100-50%
8,1590680669,Mindsets better resolution 80%
9,1590686580,Mindsets 0->40 questions


In [5]:
experiment_id = '1590676940'

In [6]:
path_in = Path(f'../output/{experiment_id}').resolve()
path_temp = Path(f'/tmp/{experiment_id}.csv').resolve()
path_out = Path(f'../plots/{experiment_id}').resolve()
path_out.mkdir(parents=True, exist_ok=True)

In [7]:
is_first_file = True
with open(path_temp,"wb") as output_file:
    for subdir, dirs, files in os.walk(path_in):
        for file in files:
            input_path = f'{subdir}/{file}'
            if is_first_file:
                is_first_file = False
                with open(input_path, "rb") as input_file:
                    output_file.write(input_file.read())
            else:
                with open(input_path, "rb") as input_file:
                    next(input_file)
                    output_file.write(input_file.read())

In [10]:
full_df = pd.read_csv(path_temp, index_col=0).reset_index(drop=True)
#full_df = full_df[full_df['nb_questions'] <= 30]

In [11]:
full_df['n_mindsets'] = full_df['mindset_sizes'].str.count(',') + 1

In [51]:
def get_mask(df, k):
    
    ps = df['p'].unique()
    ps.sort()
    ps = ps[::-1]
    sizes = df['n/k'].unique() * k
    sizes.sort()
    
    Is = np.arange(len(ps))
    Js = np.arange(len(sizes))

    mask = np.zeros((len(ps), len(sizes)), dtype=bool)

    for i, j in product(Is, Js):
        p = ps[i]
        size = sizes[j]
        n = size 
        n_k = n / k
        a = n_k / 3

        mask[i, j] = (p < 1/(k+3) and (a > p*n and a < (1-3*p) * n_k))
        
    return mask

### Plot Mindsets for 2 blocks with theoretical bounds id = 1590663805

In [12]:
full_df['size'] = full_df['mindset_sizes'].str.extract(r'(\d+),').astype(int)
full_df = full_df.rename(columns={"noise": "p", "size": "n/k"})

In [13]:
full_df['n'] = full_df['n/k'] * full_df['n_mindsets']

In [52]:
if experiment_id == '1590676940':
    experiments = [x for _, x in full_df.groupby(['nb_useless', 'n_mindsets', 'percentile_orders'])]
    for experiment in experiments:

        fig, ax = plt.subplots(figsize=(15, 10))
        
        n_mindset = experiment['n_mindsets'].iloc[0]
        name = f"Mindset_theory_n_mindsets_{n_mindset}_per_{experiment['percentile_orders'].iloc[0]}_useless_{experiment['nb_useless'].iloc[0]}"
        
        columns = ['n/k', 'p', 'Adjusted Rand Score']
        data = experiment[columns].groupby(['n/k', 'p'], as_index=False).mean()
                                                         
        make_result_heatmap(data, ax, x_column=columns[0], y_column=columns[1], values_column=columns[2], x_label='size of the clusters n/k', y_label='Noise parameter p')
        
        mask = get_mask(experiment, n_mindset)
        #mask = np.tile(mask, (1, 30))
        add_lines(mask, ax)
        ax.set_axisbelow(True)
        fig.tight_layout()

        fig.savefig(f'{path_out / name}.pdf')
        plt.close(fig) 

### Plot Mindsets for 2 blocks with theoretical bounds id = 1590432924

In [135]:
a = [x for _, x in experiment[columns].groupby(['agreement', 'noise'], as_index=False)]
a[2]

KeyError: 'agreement'

In [136]:
if experiment_id == '1590432924':
    experiments = [x for _, x in full_df.groupby(['nb_questions', 'nb_useless', 'percentile_orders'])]
    for experiment in experiments:
        
        experiment = experiment.fillna(0)
        
        
        fig, ax = plt.subplots(figsize=(15, 10))

    
        name = f"mindset_sizes_{experiment['mindset_sizes'].iloc[0]}_per_{experiment['percentile_orders'].iloc[0]}_questions_{experiment['nb_questions'].iloc[0]}_useless_{experiment['nb_useless'].iloc[0]}"
        
        columns = ['agreement', 'noise', 'Adjusted Rand Score']
        data = experiment[columns].groupby(['agreement', 'noise'], as_index=False).mean()
    
        make_result_heatmap(data, ax, x_column=columns[0], y_column=columns[1], values_column=columns[2])
        add_lines(values_theory, ax, left=False)

        ax.set_axisbelow(True)
        fig.tight_layout()

        fig.savefig(f'{path_out / name}.pdf')
        plt.close(fig) 